In [1]:
"""
Approach:
- create a bag-of-words for each of: fact1, fact2, question, answer.
- remove stopwords, as well as sentence punctuation and all verbs.
- create 2-grams using the bag-of-words

Evaluation:
- using this method, given that one of the facts states where john is, 
  we can say where he is correctly with high certainty and probability.

Limitations:
- approach only works for previously seen people and places
- if the given facts do not contain the answer to the question, 
  we don't know with certainty where the person is, as we don't keep previous facts in memory.
  
  (note the model does remember places john has been to in the past 
  based on previous examples in the training data and thus scores 
  some places more likely than others)
  
Next Steps:
- could add a 7th "don't know/can't remember" answer given low certainty, would have to relabel 
  train/test data.
  
- discover a way that the classifier can remember where the person was last
- discover a way to generalise the model to unseen people and places

"""

'\nApproach:\n- create a bag-of-words for each of: fact1, fact2, question, answer.\n- remove stopwords, as well as sentence punctuation and all verbs.\n- create 2-grams using the bag-of-words\n\nEvaluation:\n- using this method, given that one of the facts states where john is, \n  we can say where he is correctly with high certainty and probability.\n\nLimitations:\n- approach only works for previously seen people and places\n- if the given facts do not contain the answer to the question, \n  we don\'t know with certainty where the person is, as we don\'t keep previous facts in memory.\n  \n  (note the model does remember places john has been to in the past \n  based on previous examples in the training data and thus scores \n  some places more likely than others)\n  \nNext Steps:\n- could add a 7th "don\'t know/can\'t remember" answer given low certainty, would have to relabel \n  train/test data.\n  \n- discover a way that the classifier can remember where the person was last\n- dis

In [2]:
import os
import sys
%reload_ext autoreload
%autoreload
%matplotlib inline

project_root = os.path.abspath(os.path.join('../../../'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import pandas as pd
from ml_workbench.src.data_loaders.BablTask1Loader import split_parse_data

X_train, y_train = split_parse_data('/Users/enrico.t/code/ml_workbench/' + 
  'data/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt')

X_test, y_test = split_parse_data('/Users/enrico.t/code/ml_workbench/' + 
 'data/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt')

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from ml_workbench.src.vectorizers.ColumnBowVectorizer import ColumnBowVectorizer

pipeline = Pipeline([
    ('bow', ColumnBowVectorizer()),
    ('clf', SVC(probability=True))
])

parameters = {
    'clf__C': [1, 5,  10, 100, 1000],
    'clf__gamma': [10, 5, 1, 0.1, 0.001, 0.0001],
    'bow__ngrams': [[2,3]]
}

grid = GridSearchCV(
    pipeline, 
    parameters, 
    cv=3,
    n_jobs=-1, 
    verbose=1
)

grid.fit(X_train, y_train)
y_predict = grid.predict(X_test)
print grid.best_params_

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


JoblibValueError: JoblibValueError
___________________________________________________________________________
...........................................................................
/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/Users/enrico.t/code/ml_workbench/venv/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing i

In [ ]:
from pandas_ml import ConfusionMatrix

cm = ConfusionMatrix(y_test.values.flatten(), y_predict)
cm.print_stats()

In [ ]:
cm.plot()

In [ ]:
y_proba_predict = grid.predict_proba(X_test)

print X_test.iloc[0], y_test.iloc[0]
print y_proba_predict[0]

# 96% sure that john is in the hallway, as one of the given facts attests to it

In [ ]:
print X_test.iloc[1], y_test.iloc[1]
print y_proba_predict[1]

# impossible to know from the given facts where john is, low probabilities

In [ ]:
# actually a previous fact states that mary is in the bathroom, looks like the tasks follow one another.

# looks like there's also a sequence dependency between the facts, in test[3] the correct answer can only
# be known if fact 2 happened after fact 1.